# 1. Equations of state

<img style = "float: left;" src = "slides/1a_VdW.png" width = "60%">
<img style = "float: left;" src = "slides/1b_Cubic.png" width = "60%">

## 1.1. PVT behaviour of CO$_2$
In this section, we will have a look at the PVT relationship for CO2 as described by a cubic EoS.

Rather than (P,T), the EoS is explicitly expressed in terms of P(T,V).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dartsflash.pyflash import PyFlash
from dartsflash.mixtures import Mixture
from dartsflash.libflash import CubicEoS

# Define mixture and initialize Flash object
mix = Mixture(components=["CO2"])
f = PyFlash(mixture=mix)

# Add CubicEoS object
f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR))
ceos = f.eos["CEOS"]

# Define range of molar volumes for x-axis
Vmax = ceos.V(p=10, T=273.15, n=[1.])
Vmin = ceos.V(p=200, T=273.15, n=[1.])

# Define state specification, compositions and properties to evaluate
state_spec = {"volume": np.linspace(Vmin, Vmax, 1000),
              "temperature": np.array([273, 293, 304, 313]),
             }
compositions = {"CO2": 1.}
dims_order = ["volume", "temperature"]
properties = {"P": ceos.P,
              "Z": ceos.Zv,
             }

# Calculate pressure at each (T,V)
vt_props = f.evaluate_properties_1p(state_spec=state_spec, compositions=compositions,
                                    properties_to_evaluate=properties, mole_fractions=True, print_state="VT props")

# Plot P-V diagram
from dartsflash.plot import PlotEoS
pv_plot = PlotEoS.pressure_volume(f, temperatures=state_spec["temperature"], compositions=[1.],
                                  pt_props=None, vt_props=vt_props, vrange=[0, 6e-4], prange=[0, 100])
pz_plot = PlotEoS.compressibility(f, temperatures=state_spec["temperature"], compositions=[1.],
                                  pt_props=None, vt_props=vt_props, zrange=[-0.1, 1.1], prange=[-25, 200])

### <font color='Red'> Excersise: compute and plot solutions for 2 more values of p and T

## 1.2. Compressibility factor Z
In order to calculate thermodynamic properties at a given (P,T), we have to solve for volume.

<img style = "float: left;" src = "slides/1d_Z.png" width = "60%">

In [ ]:
# Find the volume roots at given (P,T)
p, t = 30, 273
ceos.solve_PT(p=p, T=t, n=[1.])
Z = np.array(ceos.Z())
Zmin, Zmax = np.amin(Z[np.imag(Z) == 0]), np.amax(Z[np.imag(Z) == 0])
R = 8.314472e-5  # m3.bar/mol.K
Vl, Vv = np.real(Zmin) * R * t / p, np.real(Zmax) * R * t / p

print(Vl, Vv)

# Evaluate the Z-factor at (Pc,Tc)
Pc, Tc = mix.comp_data.Pc[0], mix.comp_data.Tc[0]
Vc = ceos.V(p=Pc+1e-10, T=Tc, n=[1.])
Zc = ceos.Z()

pv_plot = PlotEoS.pressure_volume(f, temperatures=state_spec["temperature"], compositions=[1.],
                                  pt_props=None, vt_props=vt_props, vrange=[0, 6e-4], prange=[0, 100])
pv_plot.draw_line(x=[Vl, Vv], y=[p, p], colours=pv_plot.colours[0], linestyle="dashed")
pv_plot.draw_point(X=Vc, Y=Pc, colours=pv_plot.colours[2])

pz_plot = PlotEoS.compressibility(f, temperatures=state_spec["temperature"], compositions=[1.],
                                  pt_props=None, vt_props=vt_props, zrange=[-0.1, 1.1], prange=[-25, 200])
pz_plot.draw_line(x=[p, p], y=[np.real(Zmin), np.real(Zmax)], colours=pv_plot.colours[0], linestyle="dashed")
pz_plot.draw_point(X=Pc, Y=Zc, colours=pv_plot.colours[2])

## 1.3. Thermodynamic properties from an EoS
<img style = "float: left;" src = "slides/1e_fugacity.png" width = "60%">
<img style = "float: left;" src = "slides/1g_thermodynamic_props.png" width = "60%">

In [ ]:
# Evaluate enthalpy and entropy of CO2
state_spec = {"temperature": np.arange(248, 348, 0.1),
              "pressure": np.array([25, 50, 75, 100]),
             }
compositions = {"CO2": 1.}
properties = {"H": ceos.H_PT, "S": ceos.S_PT}

results = f.evaluate_properties_1p(state_spec=state_spec, compositions=compositions,
                                   properties_to_evaluate=properties, mole_fractions=True)


In [ ]:
# Get NIST data and plot
from ref_data import get_nist_data

R = 8.3144772

num_curves = len(state_spec["pressure"])
ref_T, ref_H, ref_S = [[] for i in range(num_curves)], [[] for i in range(num_curves)], [[] for i in range(num_curves)]

for i, p in enumerate(state_spec["pressure"]):
    nist_data = get_nist_data(component="CO2", temperature=np.arange(250, 351, 10), pressure=p, include_phase_boundary=True)
    ref_T[i] = nist_data["temperature"]
    ref_H[i] = nist_data["enthalpy"]
    ref_S[i] = nist_data["entropy"]

for i in range(num_curves):
    H0 = ref_H[i][0]
    ref_H[i][:] = [(Hj - H0) * 1e3 for Hj in ref_H[i][:]]  # set reference value of enthalpy and convert kJ/mol to J/mol
    S0 = ref_S[i][0]
    ref_S[i][:] = [(Sj - S0) * 1e3 for Sj in ref_S[i][:]]  # set reference value of entropy and convert kJ/mol to J/mol

T = [state_spec["temperature"] for i in range(num_curves)]
H0 = results.isel(pressure=0).H.values[0]  # set reference value of enthalpy at first value
H = [(results.isel(pressure=i).H.values - H0) * R for i, pres in enumerate(state_spec["pressure"])]
S0 = results.isel(pressure=0).S.values[0]  # set reference value of entropy at first value
S = [(results.isel(pressure=i).S.values - S0) * R for i, pres in enumerate(state_spec["pressure"])]
labels = ["{:.1f} bar".format(pres) for pres in state_spec["pressure"]]

from dartsflash.diagram import Plot
H_plot = Plot()
H_plot.draw_plot(xdata=T, ydata=H, logy=False, datalabels=labels)
H_plot.draw_refdata(xref=ref_T, yref=ref_H)
H_plot.add_attributes(title="Enthalpy of " + f.mixture.name, ax_labels=["temperature, K", r"$\Delta$H, J/mol"], legend=True)

S_plot = Plot()
S_plot.draw_plot(xdata=T, ydata=S, logy=False, datalabels=labels)
S_plot.draw_refdata(xref=ref_T, yref=ref_S)
S_plot.add_attributes(title="Entropy of " + f.mixture.name, ax_labels=["temperature, K", r"$\Delta$S, J/mol"], legend=True)